## Pymorphy2
Морфологический анализатор для русского языка (http://pymorphy2.readthedocs.org/).

In [4]:
import pymorphy2

def pymorphy2_311_hotfix():
    # https://github.com/pymorphy2/pymorphy2/issues/160#issuecomment-1486657176
    # https://github.com/natasha/yargy/blob/master/yargy/morph.py

    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)

pymorphy2_311_hotfix()

morph = pymorphy2.MorphAnalyzer(lang="ru")

Метод *morph.parse()* возвращает список объектов типа Parse с информацией о формах слова. Анализатор возвращает несколько вариантов разбора, так как на основе информации о том, как слово пишется  понять, какой разбор правильный, нельзя (нужен контекст).

In [5]:
def pr(lst):
    for x in lst:
        print('{}\t{}'.format(x.normal_form, x.tag))

In [6]:
pr(morph.parse('стекло'))

стекло	NOUN,inan,neut sing,nomn
стекло	NOUN,inan,neut sing,accs
стечь	VERB,perf,intr neut,sing,past,indc


В данном примере слово "стекло" может быть разобрано как глагол, как существительное в именительном падеже и сущиствительное в винительном падеже. 
Граммема - какая-то грамматическая характеристика слова (падеж, род и т.д.). Тэг - это набор граммем. В данном случае, для слова *стечь*:  
**глагол**, **совершенный вид**, **средний род**, **единственное число**, **прошедшее время**.

Полный список граммем: http://opencorpora.org/dict.php?act=gram

Как может быть разобрано слово *машина*? Примеры предложений с каждым вариантом?

In [4]:
pr(morph.parse('машина'))

машина	NOUN,inan,femn sing,nomn
машин	NOUN,anim,masc,Sgtm,Surn sing,gent
машин	NOUN,anim,masc,Sgtm,Surn sing,accs
машин	NOUN,anim,femn,Sgtm,Surn sing,nomn


Слово *дань*? Примеры?

In [5]:
pr(morph.parse('дань'))

дань	NOUN,inan,femn sing,nomn
дань	NOUN,inan,femn sing,accs
даня	NOUN,anim,masc,Name sing,voct,Infr
даня	NOUN,anim,masc,Name plur,gent
даня	NOUN,anim,masc,Name plur,accs


В случае, если слово отсутствует в словаре, разбор производится эвристически:

In [6]:
pr(morph.parse('бокренок'))

бокренок	NOUN,inan,masc sing,nomn
бокренок	NOUN,inan,masc sing,accs
бокрёнок	NOUN,anim,masc sing,nomn
бокренка	NOUN,inan,femn plur,gent


In [7]:
pr(morph.parse('на'))

на	PREP
на	PRCL
на	INTJ


In [8]:
pr(morph.parse('люди'))

человек	NOUN,anim,masc plur,nomn


### MyStem
https://api.yandex.ru/mystem/

Можно загрузить только в виде исполняемого файла, pymystem3 - обертка. Формально при разборе может учитывать контекст. 

In [9]:
from pymystem3 import Mystem
mystem = Mystem()

Installing mystem to /home/alex/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [10]:
mystem.analyze('машина')

[{'analysis': [{'lex': 'машина', 'wt': 0.999173444, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'машина'},
 {'text': '\n'}]

In [11]:
mystem.analyze('ловить глазами')

[{'analysis': [{'lex': 'ловить', 'wt': 1, 'gr': 'V,несов,пе=инф'}],
  'text': 'ловить'},
 {'text': ' '},
 {'analysis': [{'lex': 'глаз', 'wt': 1, 'gr': 'S,муж,неод=твор,мн'}],
  'text': 'глазами'},
 {'text': '\n'}]

In [12]:
mystem.analyze('Всё это стекло на помойку')

[{'analysis': [{'lex': 'весь',
    'wt': 0.5638720238,
    'gr': 'APRO=(им,мн|вин,ед,сред|им,ед,сред|вин,мн,неод)'}],
  'text': 'Всё'},
 {'text': ' '},
 {'analysis': [{'lex': 'это',
    'wt': 0.7809833731,
    'gr': 'SPRO,ед,сред,неод=(вин|им)'}],
  'text': 'это'},
 {'text': ' '},
 {'analysis': [{'lex': 'стекло',
    'wt': 0.9853860572,
    'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'стекло'},
 {'text': ' '},
 {'analysis': [{'lex': 'на', 'wt': 0.9989522965, 'gr': 'PR='}], 'text': 'на'},
 {'text': ' '},
 {'analysis': [{'lex': 'помойка', 'wt': 1, 'gr': 'S,жен,неод=вин,ед'}],
  'text': 'помойку'},
 {'text': '\n'}]

### Snowball

Стемминг - выделение основы. 
http://snowball.tartarus.org - стеммеры для разных языков. Будем использовать обертку из библиотеки NLTK.

In [13]:
from nltk.stem.snowball import RussianStemmer 
rs = RussianStemmer()

In [14]:
(rs.stem('бутявку'),
rs.stem('муки'),
rs.stem('стекло'),
rs.stem('машина'),
rs.stem('машину'))

('бутявк', 'мук', 'стекл', 'машин', 'машин')